Generate an .ann file for each .conll file in a directory.

In [1]:
import os
from brat.tools import BIOtoStandoff

In [42]:
CONLL_DIR = "/home/jan/bsc/meddoplace_jan/meddoplace_noun/noun_prediction_conlls/" # Directory with CoNLLs (with trailing '/')
TXT_DIR = "/home/jan/bsc/meddoplace_completo/test_set_reference/txt/" # Directory with .txts
ANN_DIR = "/home/jan/bsc/meddoplace_jan/meddoplace_noun/noun_prediction_anns/" # output dir

In [43]:
os.makedirs(ANN_DIR)

In [44]:
# Write an .ann file for each .conll file by calling BIOtoStandoff.py
conll_files = [file for file in os.listdir(CONLL_DIR) if file.endswith(".conll")]

for conn_file in conll_files:
    txt_file = conn_file.replace('.conll', '.txt')
    argv = ["brat/tools/BIOtoStandoff.py", TXT_DIR + txt_file, CONLL_DIR + conn_file, "-1", "0"]
    res = BIOtoStandoff.main(argv)
    ann_file = conn_file.replace('.conll', '.ann')
    with open(ANN_DIR + ann_file, 'w') as file:
        ann_content = map(lambda line: str(line)+'\n', res)
        file.writelines(ann_content)

Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" at type switch
Note: rewriting "I" -> "B" after "O"
Note: rewriting "I" -> "B" after "O"


## Merge annotation files (.ann) into one file and include filename

After merging all resulting .ann files to one using:
```bash
find . -name '*.ann' -type f -exec grep "" {} + > ../all_merged.ann
```

We then adapt the columns of the .ann file with the format required by the subtask's evaluation.
This was needed for Meddoplace evaluation.

In [37]:
MERGED_ANN = "/home/jan/bsc/meddoplace_jan/meddoplace_loc_warmup/all_merged_filename.ann"

In [38]:
import pandas as pd
import csv

In [39]:
df = pd.read_csv(MERGED_ANN, quoting=csv.QUOTE_NONE,
                 sep="\t", usecols=[0, 1, 2], names=['id', "label", 'text'], header=None)

In [40]:
df['start_span'] = df['label'].apply(lambda elem: elem.split()[1])
df['end_span'] = df['label'].apply(lambda elem: elem.split()[2])
df['label'] = df['label'].apply(lambda elem: elem.split()[0])
df['filename'] = df['id'].apply(lambda elem: elem.split(':')[0])
df['ann_id'] = df['id'].apply(lambda elem: elem.split(':')[1])
df = df.drop(columns='id')
# Reorder columns
df = df[['filename', 'ann_id', 'label', 'start_span', 'end_span', 'text']]
df['filename'] = df['filename'].apply(lambda filename: filename[2:].split('.')[0])
df

,filename,ann_id,label,start_span,end_span,text
0,casos_clinicos_profesiones5,T1,DEPARTAMENTO,44,47,MAZ
1,casos_clinicos_profesiones5,T2,FAC,82,105,Hospital Obispo Polanco
2,casos_clinicos_profesiones5,T3,GPE,109,115,Teruel
3,casos_clinicos_profesiones5,T4,FAC,277,314,Mutua de Accidentes de Zaragoza (MAZ)
4,casos_clinicos_profesiones5,T5,DEPARTAMENTO,350,375,servicio de traumatología
...,...,...,...,...,...,...
2679,caso_clinico_urgencias139,T4,FAC,518,533,centro de salud
2680,caso_clinico_urgencias139,T5,DEPARTAMENTO,2140,2151,ginecología
2681,caso_clinico_oftalmologia211,T1,GEO,54,64,zona rural
2682,caso_clinico_oftalmologia211,T2,DEPARTAMENTO,179,211,consulta provincial de Pediatría


In [41]:
df.to_csv("/home/jan/bsc/meddoplace_jan/meddoplace_loc_warmup/meddoplace_loc_warmup.tsv", quoting=csv.QUOTE_NONE, sep="\t", index=False, header=True)

### Fixing reference loc_type.tsv for meddoplace scoring script

In [26]:
df_ref = pd.read_csv("/home/jan/bsc/meddoplace_scoring_script_loc/input/ref/loc_type.tsv", sep="\t")

In [28]:
df_ref['start_span'] = df_ref['span'].apply(lambda elem: elem.split(', ')[0])
df_ref['end_span'] = df_ref['span'].apply(lambda elem: elem.split(', ')[1])
df_ref.drop(columns=['span', 'attributes', 'note'])
df_ref = df_ref[['filename', 'label', 'start_span', 'end_span', 'text']]

In [30]:
df_ref.to_csv("/home/jan/bsc/meddoplace_scoring_script_loc/input/ref/loc_type.tsv", sep="\t", index=False)